In [1]:
from typing import List
import torch
from torch import nn, optim
import numpy as np
#from pinn import PINN, IPINN

In [2]:
from typing import List, Callable, Optional

In [3]:
from plotly import graph_objects as go, io as pio
from pathlib import Path
import matplotlib.pyplot as plt

In [4]:
torch.set_default_dtype(torch.double)
torch.set_default_device(torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu"))

In [5]:
class MLP(nn.Module):
    """Multi-Layer Perceptron (MLP) module."""

    def __init__(self, layer_size: List[int], activation: nn.Module = nn.Tanh()):
        super().__init__()

        self.linear = nn.ModuleList()
        for i in range(1, len(layer_size)):
            self.linear.append(nn.Linear(layer_size[i - 1], layer_size[i]))
        self.activation = activation

    def forward(self, x):
        for i, item in enumerate(self.linear[:-1]):
            x = self.activation(item(x))
        x = self.linear[-1](x)
        return x

In [6]:
class PINN:
    """Physics-Informed Neural Network (PINN) class."""

    def __init__(self, layer_size: List[int], activation: nn.Module = nn.Tanh()):
        self.mlp = MLP(layer_size, activation)
        self.optimizer = optim.LBFGS(
            list(self.mlp.parameters()),
            lr=1,
            max_iter=1000,
            max_eval=1250 * 1000 // 15000,
            tolerance_grad=1e-8,
            tolerance_change=0,
            history_size=100,
            line_search_fn="strong_wolfe",
        )
        self.loss = nn.MSELoss()
        self.loss_history: List[float] = []
        self.epoch = 0

    def compile(
        self,
        ftns: List[Callable[[torch.Tensor, torch.Tensor], torch.Tensor]],
        pts: List[torch.Tensor],
    ):
        self.ftns = ftns
        self.pts = pts

        if not len(self.ftns) == len(self.pts):
            raise ValueError(f"Arguments `ftns` and `pts` must have the same length.")

    def train(self, epochs: int, loss_weights: Optional[List[float]] = None):
        self.mlp.train()
        if loss_weights is None:
            loss_weights = [1.0 for _ in self.ftns]
        else:
            if not len(loss_weights) == len(self.ftns):
                raise ValueError(
                    f"Arguments `loss_weights` and `ftns` must have same length."
                )

        while self.epoch < epochs:

            def closure():
                losses = []
                for i, pt in enumerate(self.pts):
                    output = self.mlp(pt)
                    losses.append(
                        loss_weights[i]
                        * self.loss(self.ftns[i](pt, output), torch.zeros_like(output))
                    )
                total_loss = sum(losses)
                self.optimizer.zero_grad()
                total_loss.backward()

                self.loss_history.append(total_loss.item())

                self.epoch += 1
                if self.epoch % 100 == 0:
                    print(f"Epoch {self.epoch}/{epochs}, Loss: {total_loss:.6f}")

                return total_loss

            self.optimizer.step(closure)

    def validation(self, exact_solution: Callable[[torch.Tensor], torch.Tensor]):
        self.mlp.eval()

        # validation points
        x = torch.linspace(0, 6, 101).reshape(-1, 1)

        # exact solution
        y = exact_solution(x)

        # evaluation
        with torch.no_grad():
            y_eval = self.mlp(x)

        # calculate mean relative L_2 norm
        # error = torch.mean(
        #     torch.norm(y - y_eval, dim=1) / (torch.norm(y, dim=1) + np.finfo(float).eps)
        # )

        # calculate relative L_2 norm
        error = torch.sqrt(
            torch.trapezoid((y - y_eval) ** 2, x, dim=0)
            / torch.trapezoid(y**2, x, dim=0)
        ).item()
        print(f"Validation Error: {error * 100:.4f} [%]")

        x = x.detach().cpu().numpy().flatten()
        y = y.detach().cpu().numpy().flatten()
        y_eval = y_eval.detach().cpu().numpy().flatten()

        train_x = torch.cat(self.pts)
        train_y = exact_solution(train_x)

        train_x = train_x.detach().cpu().numpy().flatten()
        train_y = train_y.detach().cpu().numpy().flatten()

        data = [
            go.Scatter(
                x=x,
                y=y,
                mode="lines",
                line=go.scatter.Line(width=5),
                name="True solution",
            ),
            go.Scatter(
                x=x,
                y=y_eval,
                mode="lines",
                line=go.scatter.Line(width=5, dash="dash"),
                name="Predicted solution",
            ),
            go.Scatter(
                x=train_x,
                y=train_y,
                mode="markers",
                marker=go.scatter.Marker(size=10),
                name="Training points",
            ),
        ]
        layout = go.Layout(
            template="plotly_white",
            width=1300,
            height=1300,
            font=go.layout.Font(family="Times New Roman", size=25),
        )
        fig = go.Figure(data, layout)
        print(f"Results FIle Saved in {Path.cwd().absolute()}")
        pio.write_html(fig, Path.cwd() / "forward_pinn_validation.html")

        data = [
            go.Scatter(
                y=self.loss_history,
                mode="lines",
                line=go.scatter.Line(width=5),
                name="loss",
            )
        ]
        fig = go.Figure(data, layout)
        pio.write_html(fig, Path.cwd() / "forward_pinn_loss_history.html")


In [7]:
def differential_equation(input: torch.Tensor, output: torch.Tensor) -> torch.Tensor:
    """
    Physics-Informed equation (L * d2i/dt2 + R * di/dt + 1/C * i = 0)

    ...

    Parameters
    ----------
    input : torch.Tensor
            Input tensor of shape (batch_size, 1)
    output : torch.Tensor
            Output tensor of shape (batch_size, 1)

    Returns
    -------
    Physics-Informed equation
    """

    i = output
    di_dt = derivative(input, output)
    d2i_dt2 = second_derivative(input, output)

    return L * d2i_dt2 + R * di_dt + i / C

def initial_condition(input: torch.Tensor, output: torch.Tensor) -> torch.Tensor:
    """
    Initial condition (i(0) = 0)

    ...

    Parameters
    ----------
    input : torch.Tensor
            Input tensor of shape (the number of initial points, 1)
    output : torch.Tensor
            Output tensor of shape (the number of initial points, 1)

    Returns
    -------
    Initial value at t = 0
    """

    i = output
    return i

def initial_condition2(input: torch.Tensor, output: torch.Tensor) -> torch.Tensor:
    """
    Initial condition of derivative (L * di/dt = V0)

    ...

    Parameters
    ----------
    input : torch.Tensor
            Input tensor of shape (the number of initial points, 1)
    output : torch.Tensor
            Output tensor of shape (the number of initial points, 1)

    Returns
    -------
    Initial value of derivative at t = 0
    """

    di_dt = derivative(input, output)
    return L * di_dt - V0

In [8]:
def derivative(input: torch.Tensor, output: torch.Tensor) -> torch.Tensor:
    """
        Calculate the derivative of `output` with respect to `input`.

        ...

        Parameters
        ----------
        input : torch.Tensor
            Input tensor of shape (batch_size, 1)
        output : torch.Tensor
            Output tensor of shape (batch_size, 1)

        Returns
        -------
        Derivative of `output` with respect to `input`.
    """

    return torch.autograd.grad(output, input, grad_outputs=torch.ones_like(output), create_graph=True)[0]

def second_derivative(input: torch.Tensor, output: torch.Tensor) -> torch.Tensor:
    """
    Calculate the second derivative of `output` with respect to `input`.

    ...

    Parameters
    ----------
    input : torch.Tensor
            Input tensor of shape (batch_size, 1)
    output : torch.Tensor
            Output tensor of shape (batch_size, 1)

    Returns
    -------
    Second derivative of `output` with respect to `input`.
    """

    deri = derivative(input, output)
    return torch.autograd.grad(deri, input, grad_outputs=torch.ones_like(deri), create_graph=True)[0]

In [9]:
R = 1.2
L = 1.5
C = 0.3
V0 = 12.0

def exact_solution(input: torch.Tensor) -> torch.Tensor:
    """
    Exact solution

    ...

    Parameters
        ----------
    input : torch.Tensor

    Returns
    -------
    Exact solution
    """

    t = input
    return 5.57 * torch.exp(-0.4 * t) * torch.sin(1.44 * t)

In [10]:
# the number of points for training
n_points = 41

# Construct training points randomly in [0, 1]
x = (6*torch.rand((n_points, 1))).requires_grad_(True)

# Boundary points (x = 0)
bp = torch.zeros((1, 1)).requires_grad_(True)

# Differential equations and boundary value functions
ftns = [differential_equation, initial_condition, initial_condition2]

# Points list corresponding to ftns
pts = [x, bp, bp]

pinn = PINN(layer_size=[1, 10, 10, 10, 1],activation=torch.nn.Tanh(),)
pinn.compile(ftns=list(ftns), pts=list(pts))

pinn.train(epochs=1500)
pinn.validation(exact_solution)

Epoch 100/1500, Loss: 0.002323
Epoch 200/1500, Loss: 0.000053
Epoch 300/1500, Loss: 0.000009
Epoch 400/1500, Loss: 0.000000
Epoch 500/1500, Loss: 0.000000
Epoch 600/1500, Loss: 0.000000
Epoch 700/1500, Loss: 0.000000
Epoch 800/1500, Loss: 0.000000
Epoch 900/1500, Loss: 0.000000
Epoch 1000/1500, Loss: 0.000000
Epoch 1100/1500, Loss: 0.000000
Epoch 1200/1500, Loss: 0.000000
Epoch 1300/1500, Loss: 0.000000
Epoch 1400/1500, Loss: 0.000000
Epoch 1500/1500, Loss: 0.000000
Validation Error: 99.9990 [%]
Results FIle Saved in /content
